In [17]:
import json
import pandas as pd
import numpy as np
from collections import OrderedDict
from itertools import groupby
from datetime import datetime
from pymongo import MongoClient

In [2]:
# Connect
client = MongoClient("mongodb://mongo:27017")

# Connect to `nywater` database
db = client['nywater']

In [75]:
# Import water_sampling.csv
water_sampling_df = pd.read_csv('water_sampling.csv')
water_sampling_df = water_sampling_df.replace(np.nan, 0)

In [69]:
water_sampling_df.head()

,School District,School,County,Type of Organization,Number of Outlets,Any Buildings with Lead-Free Plumbing?,Previously Sampled Outlets,Outlets Waiver Requested,Waivers Granted,Outlets Sampled After Regulation,...,School Website,BEDS Code,School Street,School City,School State,School Zip Code,Date Sampling Updated,Date Results Updated,County Location,Location
0,ALBANY CITY SCHOOL DISTRICT,WEST HILL MIDDLE SCHOOL,Albany,Public School,50.0,No,0.0,0.0,0.0,50.0,...,http://www.albanyschools.com/,010100010046,NaN,NaN,NaN,NaN,11/16/2016,12/07/2016,"(42.678066, -73.814233)",NaN
1,BOCES No District Name,ORLEANS NIAGARA BOCES SANBORN WEST NIAGARA BLD...,Orleans,BOCES School,NaN,No,NaN,NaN,NaN,NaN,...,NaN,459000000007,NaN,NaN,NaN,NaN,NaN,NaN,"(43.248394, -78.218438)",NaN
2,BOCES No District Name,WESTCHESTER BOCES RYE LAKE BUILDING G (LEAD ONLY),Westchester,BOCES School,NaN,No,NaN,NaN,NaN,NaN,...,NaN,669000000048,NaN,NaN,NaN,NaN,NaN,NaN,"(41.039278, -73.805386)",NaN
3,BUFFALO CITY SCHOOLS,PS 200 BENNETT HIGH SCHOOL,Erie,Public School,161.0,No,0.0,0.0,0.0,161.0,...,http://www.buffaloschools.org,140600010099,NaN,NaN,NaN,NaN,01/20/2017,01/20/2017,"(42.888143, -78.795544)",NaN
4,CENTRAL SQUARE CENTRAL SCHOOL,CENTRAL SQUARE INTERMEDIATE SCHOOL,Oswego,Public School,87.0,No,32.0,0.0,0.0,50.0,...,http://www.cssd.org/,460801060003,NaN,NaN,NaN,NaN,11/08/2016,11/08/2016,"(43.39123, -76.31133)",NaN


In [32]:
# Sanitizes school name
def sanitizeName(name):
    name = str(name).strip().title().replace(' (Lead Only)', '')
    name = name.replace('H S', 'High School')
    name = name.replace('E S', 'Elementary School')
    name = name.replace('Es', 'Elementary School')
    name = name.replace('Hs', 'High School')
    name = name.replace('Ms', 'Middle School')
    name = name.replace('Elem School', 'Elementary School')
    name = name.replace('P S ', 'PS #')
    
    return name

In [73]:
# Stores all schools
results = dict()

# Logs
print('Processing water_sampling.csv...')

# Iterates over each row in the water sampling dataframe
for row in water_sampling_df.iterrows():
    index, data = row
    
    # Defines newSchool
    # TODO - add more attributes here...
    newSchool = dict({
        'name': sanitizeName(data['School']),
        'beds_code': str(data['BEDS Code']).strip().title(),
        'district': str(data['School District']).strip().title(),
        'address': {
            #'street': str(data['School Street']).strip().title(),
            'city': str(data['School City']).strip().title(),
            'state': str(data['School State']).strip().title(),
            'county': str(data['County']).strip().title(),
            'zip_code': str(int(data['School Zip Code'])).strip().title()
        },
        'sector': str(data['Type of Organization']).strip().title(),
        'outlets': str(data['Number of Outlets']).strip().title(),
        'lead_free': str(data['Any Buildings with Lead-Free Plumbing?']).strip().title(),
        'outlets_sampled_pre_2016': str(data['Previously Sampled Outlets']).strip().title(),
        'outlet_waivers_requested': str(data['Outlets Waiver Requested']).strip().title(),
        'outlet_waivers_granted': str(data['Waivers Granted']).strip().title(),
        'outlets_sampled_post_regulation': str(data['Outlets Sampled After Regulation']).strip().title(),
        'sampling_completed': str(data['Sampling Complete']).strip().title(),
        'sampling_completed_date': str(data['Sampling Completion Date']).strip().title(),
        'outlets_leq_15ppb': str(data['Number of Outlets, Result ≤ 15 ppb']).strip().title(),
        'outlets_gt_15ppb': str(data['Number of Outlets, Result > 15 ppb']).strip().title(),
        'out_of_service': str(data['Out of Service']).strip().title(),
        #'all_results_received': str(data['All Results Received']).strip().title(),
        'all_results_received_date': str(data['Date All Results Received']).strip().title(),
        #'website': data['School Website'],
        'sampling_update_date': str(data['Date Sampling Updated']).strip().title(),
        'results_update_date': str(data['Date Results Updated']).strip().title(),
        'county_coords': str(data['County Location']).strip().title(),
        'coords': str(data['Location']).strip().title().replace('(', '').replace(')', '').split(', ')
    })
    
    
    # Adds the school to our complete list
    results[str(data['BEDS Code']).strip()] = newSchool

# Logs
print('Done processing Water Sampling...')

Processing water_sampling.csv...


ValueError: cannot convert float NaN to integer

In [92]:
# import matplotlib.pyplot as plt
# pd.options.display.mpl_style = 'default'
# water_sampling_df.hist()

In [40]:
# results

In [38]:
len(results)

4662

In [71]:
# Generates some statistics
# water_sampling_df["Number of Outlets, Result > 15 ppb"].unique()
# water_sampling_df["School Website"].unique()

In [57]:
# Iterates over each School and inserts it into Mongo
for k, v in results.items():
    db['schools'].insert_one(v)
#     print(json.dumps(v, indent=2)) 

# Logs..
print('Wrote Schools to MongoDb...')

Wrote Schools to MongoDb...
